In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy.spatial import distance

Your task is to modify `KNNClassifier` class from your practice in class. The `KNNClassifier` class with empty methods is provided below. Please, modify it to do all tasks.

In [13]:
# playground

points_labels = [[0, 0], [1, 0]]
tree = cKDTree(points_labels)

i = tree.query_ball_point([1.1, 0], 2)
tree.data[i]

array([[0., 0.],
       [1., 0.]])

In [4]:
from scipy.spatial import cKDTree


class KNNClassifier(object):
    
    def __init__(self, max_dist=1., use_kd_tree=False, leafsize=30, use_weights=False):
        """
        This is a constructor of the class. 
        Here you can define parameters (max_dist) of the class and 
        attributes, that are visible within all methods of the class.
        
        Parameters
        ----------
        max_dist : float
            Maximum distance between an object and its neighbors.
        """
        
        # Make this parameter visible in all methods of the class
        self.max_dist = max_dist
        self.use_kd_tree = use_kd_tree
        self.leafsize = leafsize
        if use_weights:
            self.weights_func = lambda point_a, point_b: 1 / self.dist(point_a, point_b)
        else:
            self.weights_func = lambda point_a, point_b: 1
                
    
    def fit(self, X, y):
        """
        This method trains the KNN classifier. 
        Actualy, the KNN classifier has no training procedure.
        It just remembers data (X, y) that will be used for predictions.
        
        Parameters
        ----------
        X : numpy.array, shape = (n_objects, n_features)
            Matrix of objects that are described by their input features.
        y : numpy.array, shape = (n_objects)
            1D array with the object labels. 
            For the classification labels are integers in {0, 1, 2, ...}.
        """
        
        ### Your code here
        self.labels = list(set(y))
        if self.use_kd_tree:
            self.kd_tree = cKDTree(X, leafsize=self.leafsize)
            self.points_2_labels = {}
            for i in range(X.shape[0]):
                point = X[i]
                label = y[i]
                self.points_2_labels[tuple(point)] = label    
        else:
            self.X = X
            self.y = y

    def dist(self, point_a, point_b):
        return distance.euclidean(point_a, point_b)
            
    def do_predict(self, new_point):
        if self.use_kd_tree:
            close_points = self.kd_tree.data[self.kd_tree.query_ball_point(new_point, self.max_dist)]
            close_labels = []
            gist = {}
            for point in close_points:
                label = self.points_2_labels[tuple(point)]
                close_labels.append(label)
                if label not in gist:
                    gist[label] = 0
                gist[label] += self.weights_func(new_point, point)
            return gist
        
        gist = {}
        n = self.X.shape[0]
        for i in range(n):
            point = self.X[i]
            label = self.y[i]
            if self.dist(new_point, point) < self.max_dist:
                if label not in gist:
                    gist[label] = 0
                gist[label] += self.weights_func(new_point, point)
        return gist

    def get_label(self, new_point):
        data = self.do_predict(new_point)
        if len(data) == 0:
            return self.labels[0]
        else:
            mx_key = self.labels[0]
            mx_value = 0
            for key in data:
                value = data[key]
                if value > mx_value:
                    mx_value = value
                    mx_key = key
            return mx_key
    
    def get_probabilities(self, new_point):
        gist = self.do_predict(new_point)
        close_object_values = sum(gist.values())
        probabilities = []
        for label in self.labels:
            if label not in gist:
                probabilities.append(0.0)
            else:
                probabilities.append(gist[label] / close_object_values)
        return probabilities
        
    def predict(self, X):
        """
        This methods performs labels prediction for new objects.
        
        Parameters
        ----------
        X : numpy.array, shape = (n_objects, n_features)
            Matrix of objects that are described by their input features.
            
        Returns
        -------
        y_predicted : numpy.array, shape = (n_objects)
            1D array with predicted labels. 
            For the classification labels are integers in {0, 1, 2, ...}.
        """
        
        # Create an empty list for predicted labels
        y_predicted = []
        
        ### Replace this line with your code:
        
        for new_point in X:
            y_predicted.append(self.get_label(new_point))
        
        ### The end of your code
            
        return np.array(y_predicted) # return numpy.array
    
    
    def predict_proba(self, X):
        """
        This methods performs prediction of probabilities of each class for new objects.
        
        Parameters
        ----------
        X : numpy.array, shape = (n_objects, n_features)
            Matrix of objects that are described by their input features.
            
        Returns
        -------
        y_predicted_proba : numpy.array, shape = (n_objects, n_classes)
            2D array with predicted probabilities of each class. 
            Example:
                y_predicted_proba = [[0.1, 0.9],
                                     [0.8, 0.2], 
                                     [0.0, 1.0], 
                                     ...]
        """
        
        # Create an empty list for predictions
        y_predicted_proba = []
        
        ### Replace these lines with your code:
        
        for new_point in X:
            y_predicted_proba.append(self.get_probabilities(new_point))
        
        ### The end of your code
            
        return np.array(y_predicted_proba) # return numpy.array
    
# x = np.array([[0, 2],
#               [0, -2],
#               [-1, 0],
#               [1, 0]])
# y = ['y', 'y', 'x', 'x']

# model = KNNClassifier(3, use_weights=True)
# model.fit(x, y)

# model.predict([[0, 0]])
# model.predict_proba([[100, 100]])

### Task 1 (1 point) <br/>
Create a matrix of object features `X` and vector of labels `y` for N=1000 objects using `sklearn.datasets.make_moons()` function from scikit-learn library. Also, set up random state in the function `random_state=42` and `noise=0.2`. To open the function description use `Shift` + `Tab` .

In [5]:
from sklearn import datasets

In [6]:
X, y = datasets.make_moons(1000)
X.shape


(1000, 2)

In [7]:
### Your code here
X, y = datasets.make_moons(n_samples=1000, random_state=42, noise=0.2)

### Check your solution
ans = np.array([[-0.112,  0.52 ],
                [ 1.143, -0.343]])
assert np.array_equal(np.round(X[:2], 3), ans), ('Check your solution.')

### Task 2 (1 point) <br/>

Split the sample into train and test samples using `sklearn.model_selection.train_test_split()` function from scikit-learn library. Use `random_state = 42` and `test_size = 0.5`.

In [8]:
from sklearn import model_selection

### Your code here
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, train_size=0.5, random_state=42)


### Check your solution
ans = np.array([[ 0.77 , -0.289],
                [ 0.239,  1.041]])
assert np.array_equal(np.round(X_train[:2], 3), ans), ('Check your solution.')

### Task 3 (2 points) <br/>

Modify class `KNNClassifier` above and implement `predict()` method that uses **max_dist** parameter to select neighbors like it's shown in the second figure (radius search). If there is no any object within **max_dist**, make decision based on the closest neighbor.

<img src="img/knn2.png" width="600">

In [9]:
# Create a class object
knn = KNNClassifier(max_dist=0.5, use_kd_tree=False)

# Train the classifier
knn.fit(X_train, y_train)

# Make prediction using the trained classifier
%time y_test_predict = knn.predict(X_test) # measure time for prediction

# Import accuracy_score function
from sklearn.metrics import accuracy_score
# Calculate accuracy for the test sample
accuracy_test = accuracy_score(y_test, y_test_predict)
print("Test accuracy of KNN classifier: ", accuracy_test)


### Check your solution
assert accuracy_test == 0.964, ('Check your solution.')

CPU times: user 2.67 s, sys: 5.16 ms, total: 2.67 s
Wall time: 2.67 s
Test accuracy of KNN classifier:  0.964


### Task 4 (2 points) <br/>

There are an algorithm [kd-tree](https://en.wikipedia.org/wiki/K-d_tree) that helps to find neighbors faster. Using [scipy.spatial.cKDTree](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.spatial.cKDTree.html#scipy.spatial.cKDTree) function modify you classifier to speed up **predict** method. Use `leafsize=30` in `KDTree`. Similar to `max_dist` option, add option `use_kd_tree = True/False` to your classifier.

In [10]:
# Create a class object
knn = KNNClassifier(max_dist=0.5, use_kd_tree=True)

# Train the classifier
knn.fit(X_train, y_train)

# Make prediction using the trained classifier
%time y_test_predict = knn.predict(X_test) # measure time for prediction

# Import accuracy_score function
from sklearn.metrics import accuracy_score
# Calculate accuracy for the test sample
accuracy_test = accuracy_score(y_test, y_test_predict)
print("Test accuracy of KNN classifier: ", accuracy_test)

### Check your solution
assert accuracy_test == 0.964, ('Check your solution.')

CPU times: user 83.9 ms, sys: 1.52 ms, total: 85.5 ms
Wall time: 84.3 ms
Test accuracy of KNN classifier:  0.964


### Task 5 (3 points) <br/>

Now modify the **predict** method to provide prediction with neighbors weights.

<img src="img/wv1.png">

<img src="img/wv2.png">

We propose you to use the following weights:

$$
w_{i} = \frac{1}{\rho(x, x_{i})}
$$

Similar to `max_dist` option, add option `use_weights = True/False` to your classifier.

In [11]:
# Create a class object
knn = KNNClassifier(max_dist=0.5, use_kd_tree=True, use_weights=True)

# Train the classifier
knn.fit(X_train, y_train)

# Make prediction using the trained classifier
%time y_test_predict = knn.predict(X_test) # measure time for prediction

# Import accuracy_score function
from sklearn.metrics import accuracy_score
# Calculate accuracy for the test sample
accuracy_test = accuracy_score(y_test, y_test_predict)
print("Test accuracy of KNN classifier: ", accuracy_test)



### Check your solution
assert accuracy_test == 0.968, ('Check your solution.')

CPU times: user 476 ms, sys: 2.82 ms, total: 479 ms
Wall time: 477 ms
Test accuracy of KNN classifier:  0.968


### Task 6 (3 points) <br/>

Develop **predict_proba** method of the classifier. For each object this method returns probability that the object belongs to each of the classes. 

For each object $x$ probability for each class is defined as:

$$
p_{c}(x) = \frac{g_{c}(x)}{\sum_{i=1}^{C} g_{i}(x)}
$$

where $C$ is number of classes.

Then, the object has a vector of probabilities:

$$
p(x) = (p_{1}(x), p_{2}(x), ..., p_{C}(x))
$$

Use neighbors weights as in Task 5.

In [12]:
# Create a class object
knn = KNNClassifier(max_dist=0.5, use_kd_tree=False, use_weights=True)

# Train the classifier
knn.fit(X_train, y_train)

# Make prediction using the trained classifier
%time y_test_predict_proba = knn.predict_proba(X_test) # measure time for prediction

# Example of the output
y_test_predict_proba[:10, :] # the first 10 rows

### Check your solution
ans = np.array([[0.046, 0.954],
                [0.962, 0.038]])
assert np.array_equal(np.round(y_test_predict_proba[:2], 3), ans), ('Check your solution.')

CPU times: user 3.04 s, sys: 6.24 ms, total: 3.05 s
Wall time: 3.05 s


### Check out the architecture of class, I just change do_predict method to modify my model for different tasks.
### The most flexible (not really) kNN ever  :)